## Setup and Import Libraries

In [1]:
import os
import json
from langchain_openai.llms import OpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.base import RunnableMap
from langchain_community.vectorstores import DocArrayInMemorySearch
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv()

True

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
model = ChatOpenAI()

In [5]:
embeddings = OpenAIEmbeddings()

In [6]:
output_parser = StrOutputParser()

## Simple Chain

In [7]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)

In [8]:
chain = prompt | model | output_parser

In [9]:
chain.invoke({"topic": "dog"})

"Why did the dog sit in the shade? \n\nBecause he didn't want to be a hot dog!"

## Complex Chain

And Runnable Map to supply user-provided inputs to the prompt.

In [13]:
vectorstore = DocArrayInMemorySearch.from_texts(
    texts=["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)

In [14]:
retriever = vectorstore.as_retriever()

In [16]:
retriever.get_relevant_documents("where did harrison work?")

[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [17]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='harrison worked at kensho')]

In [18]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [19]:
runnable = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [20]:
chain = runnable | prompt | model | output_parser

chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [21]:
inputs = runnable = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Binding OpenAI Function

In [22]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
]

In [23]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)

In [24]:
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [25]:
runnable = prompt | model

In [26]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 64, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BtU9pahXT68RBIlH9ycQw8I1ydURM', 'finish_reason': 'function_call', 'logprobs': None}, id='run--71a6d277-ae74-4a1b-861e-fbce308cd209-0', usage_metadata={'input_tokens': 64, 'output_tokens': 16, 'total_tokens': 80, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [27]:
functions = [
    {
        "name": "weather_search",
        "description": "Search for weather given an airport code",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "The airport code to get the weather for"
                },
            },
        "required": ["airport_code"]
        }
    },
    {
        "name": "sports_search",
        "description": "Search for news of recent sport events",
        "parameters": {
            "type": "object",
            "properties": {
                "team_name": {
                    "type": "string",
                    "description": "The sports team to search for"
                },
            },
            "required": ["team_name"]
        }
    }
]

In [28]:
model = model.bind(functions=functions)

In [29]:
runnable = prompt | model

In [30]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"patriots"}', 'name': 'sports_search'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 99, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BtU9s7EKCHWncvNhoeemyozDCOOvJ', 'finish_reason': 'function_call', 'logprobs': None}, id='run--697be2ce-647b-477a-9c97-aa2686a89895-0', usage_metadata={'input_tokens': 99, 'output_tokens': 18, 'total_tokens': 117, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Fallbacks

In [31]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="gpt-3.5-turbo-instruct"
)

In [32]:
simple_chain = simple_model | json.loads

In [33]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [34]:
simple_model.invoke(challenge)

'\n\n{\n    "title": "Autumn Leaves",\n    "author": "Emily Dickinson",\n    "first_line": "The leaves are falling, one by one"\n}\n\n{\n    "title": "The Ocean\'s Song",\n    "author": "Pablo Neruda",\n    "first_line": "I hear the ocean\'s song, a symphony of waves"\n}\n\n{\n    "title": "A Winter\'s Night",\n    "author": "Robert Frost",\n    "first_line": "The snow falls softly, covering the ground"\n}'

### Note: The next line is expected to fail.

In [35]:
simple_chain.invoke(challenge)

JSONDecodeError: Extra data: line 9 column 1 (char 125)

In [36]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [37]:
chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

In [38]:
final_chain = simple_chain.with_fallbacks([chain])

final_chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

## Interface

In [39]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)

model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [40]:
chain.invoke({"topic": "dogs"})

"Why do dogs run in circles before lying down? Because it's too hard to make a square!"

In [41]:
chain.batch([{"topic": "dogs"}, {"topic": "frogs"}])

["Why don't dogs make good dancers? \n\nBecause they have two left feet!",
 'Why did the frog take the bus to work? Because his car got toad away!']

In [42]:
for t in chain.stream({"topic": "dogs"}):
    print(t)


Why
 do
 dogs
 run
 in
 circles
 before
 lying
 down
?
 


Because
 it
's
 too
 hard
 to
 make
 a
 square
 bed
 with
 round
 p
aws
!



In [43]:
response = await chain.ainvoke({"topic": "dogs"})
response

'Why do dogs run in circles before lying down? \nBecause they\'re trying to make a "ruff" impression!'